In [1]:
using Plots
using QuadGK
using NLopt
#using SpecialFunctions
using NLsolve
using Statistics
include("../../scaling_CGS.jl")

print_all_CGS (generic function with 2 methods)

In [2]:
#---------- Double current sheet setup ----------
base = BaseUnits("CGS")

electron_mass_scaling = 1e0#4.590381684400168
charge_scaling = 1e0
eps_0_scaling = 1e0#1e2
mu_0_scaling = 1e0

scale_base_units(base, eps_0_scaling, mu_0_scaling, electron_mass_scaling, charge_scaling)

mass_density_scale = 1.6735328363560001e-15
length_scale       = 720.0847332647692 # proton skin depth
time_scale         = 3.5219920728365707e-9
temperature_scale  = 1e0


length = 720.0847332647692
number_density=1e9 
B_flux=10
temperature   = 1e6
temperature_e = temperature
temperature_p = temperature



scaling = ScalingCGS(base,
                number_density, length, temperature, temperature_e, temperature_p, B_flux,
                length_scale, mass_density_scale, time_scale, temperature_scale)

ds = 0.001953125
per_cell = 32

set_macro_particle_weights(scaling, ds, per_cell);
print_all_CGS(scaling)


#scaling.p_gyro_radiues_real / scaling.debye_len_real


 Maxwell Lorents Factors used in code:
 k_E =                                             =  1.0000e+00 
 k_B =                                             =  6.8198e+00 
 k_F =                                             =  6.8198e+00 
 k_D =                                             =  1.0000e+00 
 k_M =                                             =  1.4663e-01 
 k_H =                                             =  1.0000e+00 

 Macro particle weights:
                          ds =                     =  1.9531e-03 
                    per_cell =                     =  3.2000e+01 
               weight_scaled =                     =  8.6934e+07 
 weight_scaled_mass_electron =                     =  1.2673e-13 
   weight_scaled_mass_proton =                     =  2.3270e-10 
        weight_scaled_charge =                     =  9.6281e-12 
                 rho_n_ratio =                     =  3.7338e+17 

 Fundamental physical constants:
        Gravitational Constant [ cm^3 g^-1

In [3]:
scaling.c_code / scaling.v_a_code

434.7543167005872

In [4]:
30 / dt

UndefVarError: UndefVarError: dt not defined

In [5]:
println(scaling.mass_density_code * scaling.rho_n_ratio)
println(scaling.mass_density_code * scaling.rho_n_ratio * ds^3)
println(scaling.mass_density_code * scaling.rho_n_ratio * ds^3 / scaling.weight_scaled)

3.7337979268218586e17
2.7818962386413355e9
32.0


In [6]:
(scaling.proton_plasma_freq_code * scaling.debye_len_code)# / scaling.c_code

4.4437277898443394e-5

In [7]:
dx = 0.2
area = dx^2
dt = 4.7354828612241809E-004
weight_flux_per_dt = 0.075 
weight_flux = weight_flux_per_dt  / dt
number_flux = weight_flux * scaling.weight_scaled
mass_flux_e = number_flux * scaling.m_e_code
mass_flux_p = number_flux * scaling.m_p_code

3.685536131639314e-8

In [8]:

nx = 32
ng = 3
lb = 1
li = lb + ng
ui = li + nx - 1
ub = ui + ng

println("lb = ", lb)
println("li = ", li)
println("ui = ", ui)
println("ub = ", ub)

lb = 1
li = 4
ui = 35
ub = 38


In [9]:
gamma = 1.4
println("mass density code", scaling.mass_density_code)
println("rho_n_ratio", scaling.rho_n_ratio)
println("k_b code", scaling.k_B_code)
println("T code", scaling.temperature_code)
println("gamma = ", gamma)
println("gamma - 1 = ", gamma - 1)

mass density code1.0
rho_n_ratio3.7337979268218586e17
k_b code5.2857622487510206e-33
T code1.0e6
gamma = 1.4
gamma - 1 = 0.3999999999999999


In [41]:
gamma = 1.4
e_code = 2 * scaling.mass_density_code * scaling.rho_n_ratio * scaling.k_B_code * scaling.temperature_code / (gamma - 1.0)
e_code_mass = e_code / scaling.mass_density_code

e_real = e_code * scaling.energy_per_volume_scaling
e_real_mass = e_code_mass * scaling.energy_per_mass_scaling

P_code = e_code * (gamma - 1)
println("e_real = ", e_real)
println("e_code = ", e_code)
println("e_code_mass = ", e_code_mass)
println("e_real_mass = ", e_real_mass)

e_real = 0.6903242600000002
e_code = 9.867984063029905e-9
e_code_mass = 9.867984063029905e-9
e_real_mass = 4.1249519878147875e14


In [53]:
B_e = (7.20264779E-07 - 9.86790383E-09) * 2

1.42079375034e-6

In [65]:
1.20344071E-03^2 #- 1.54810550E-07^2 + 6.00548265E-06^2

1.4482695424853042e-6

In [13]:
println("mass density code", scaling.mass_density_code)
println("rho_n_ratio ", scaling.rho_n_ratio)
println("k_b code ", scaling.k_B_code)
println("T code ", scaling.temperature_code)
println("gamma = ", gamma)
println("gamma - 1 = ", gamma - 1)

mass density code1.0
rho_n_ratio 3.7337979268218586e17
k_b code 5.2857622487510206e-33
T code 1.0e6
gamma = 1.4
gamma - 1 = 0.3999999999999999


In [14]:
println("mass density code * w_rp", scaling.mass_density_code * scaling.rho_n_ratio)
println("mass density code * w_rp * k_B ", scaling.mass_density_code * scaling.rho_n_ratio * scaling.k_B_code)
println("mass density code * w_rp * k_B * T ", scaling.mass_density_code * scaling.rho_n_ratio * scaling.k_B_code *  scaling.temperature_code)


mass density code * w_rp3.7337979268218586e17
mass density code * w_rp * k_B 1.9735968126059804e-15
mass density code * w_rp * k_B * T 1.9735968126059804e-9


In [15]:
dt = 3.3300281692659984E-003
area = ds^2 * 32


proton_electron_mass_ratio = sqrt(scaling.m_p_code / scaling.m_e_code)
mom_e_frac = 1 / (proton_electron_mass_ratio + 1)
mom_p_frac = proton_electron_mass_ratio * mom_e_frac


rho = scaling.mass_density_code
u = scaling.v_thermal_e_code / 10
mom = u * rho * mom_e_frac




mom_flux = u^2 * rho * mom_e_frac
mom_flux = mom_flux * area * dt

mass_flux = u * rho * area * dt 
number_flux = mass_flux * scaling.rho_n_ratio
weight_flux = number_flux / scaling.weight_scaled


E_tot = e_code #/ 2 # assume half energy in proton and half in electrons
E_tot = E_tot + 0.5 * u^2 * rho
P_species = P_code  # assume half energy in proton and half in electrons
energy = u * (E_tot + P_species)
#energy = energy / 2. # assume half energy in proton and half in electrons
energy_flux = energy * area * dt 

println("dt = ", dt, " area = ", area)
println("mass_flux = ", mass_flux)
println("number_flux = ", number_flux)
println("weight_flux = ", weight_flux)
println("mom_flux = ", mom_flux)
println("energy_flux = ", energy_flux)

dt = 0.0033300281692659984 area = 0.0001220703125
mass_flux = 7.7403358650566e-11
number_flux = 2.890085000585321e7
weight_flux = 0.33244489400473964
mom_flux = 3.3611551757012376e-16
energy_flux = 2.4725861242636352e-18


In [20]:
# Boundary:

const_u = scaling.v_thermal_e_code / 10
const_rho = scaling.mass_density_code
const_mom = const_u * const_rho 
const_e = const_rho * scaling.rho_n_ratio * scaling.k_B_code * scaling.temperature_code / (gamma - 1.0)
const_e = const_e * 2
const_e = const_e + 0.5 * const_u^2 * const_rho
E_tot = const_e
E_internal = E_tot - 0.5 *  const_rho * const_u^2
P_tot = E_internal * (gamma - 1)
E_bulk = 0.5 * const_rho * const_u^2
println("const u / 1e-4 = ", const_u / 1e-4)
println("Total enegy = ", E_tot)
println("Internal enegy = ", E_internal)
println("Pressure = ", P_tot)
println("Bulk enegy = ", E_bulk)


const u / 1e-4 = 1.9041530036221939
Total enegy = 2.799697736904702e-8
Internal enegy = 9.867984063029903e-9
Pressure = 3.947193625211961e-9
Bulk enegy = 1.8128993306017116e-8


In [21]:
E_flux = const_u * E_tot  # Adding P_tot makes the temperature too high... But I need to do it according to the energy equation
#E_flux = const_u * (E_tot + P_tot) # Adding P_tot makes the temperature too high... But I need to do it according to the energy equation
#E_flux = const_u * (gamma * E_tot - (gamma - 1)* 0.5 * const_u^2 * const_rho ) # Adding P_tot makes the temperature too high... But I need to do it according to the energy equation
mom_flux = const_u^2 * const_rho #+ P_tot
mass_flux = const_u * const_rho
number_flux = mass_flux * scaling.rho_n_ratio
weight_flux = number_flux / scaling.weight_scaled
println("mass_flux = ", mass_flux)
println("number_flux = ", number_flux)
println("weight_flux = ", weight_flux)
println("mom_flux = ", mom_flux)
println("E_flux = ", E_flux)
println("----- Now scaling with area and dt -----")
E_flux_dt = E_flux * area * dt
mom_flux_dt = mom_flux * area * dt
mass_flux_dt = mass_flux * area * dt
number_flux_dt = number_flux * area * dt
weight_flux_dt = weight_flux * area * dt
println("mass_flux = ", mass_flux_dt)
println("number_flux = ", number_flux_dt)
println("weight_flux = ", weight_flux_dt)
println("mom_flux = ", mom_flux_dt)
println("E_flux = ", E_flux_dt)

mass_flux = 0.0001904153003622194
number_flux = 7.109722537276162e13
weight_flux = 817827.4877137493
mom_flux = 3.625798661203423e-8
E_flux = 5.331052854961347e-12
----- Now scaling with area and dt -----
mass_flux = 7.7403358650566e-11
number_flux = 2.890085000585321e7
weight_flux = 0.33244489400473964
mom_flux = 1.4738783786492117e-14
E_flux = 2.167060080428126e-18


In [22]:

mesh_bulk_dt = 0.5 * const_u * const_mom^2 / const_rho * area* dt
mesh_bulk_dt_elec = mesh_bulk_dt * mom_e_frac



mesh_internal_dt = 0.5 * (E_flux_dt - mesh_bulk_dt)

3.819075547943865e-19

In [23]:
E_internal

9.867984063029903e-9

In [24]:
E_internal = 1.77251074E-17 - 3.76384632E-18
w_part = 7.31375933
n_part = w_part * scaling.weight_scaled


E_internal = E_internal / 10
T = (gamma - 1.) * E_internal / (n_part * scaling.k_B_code)

166167.23792113594

In [395]:
μ = 10 #scaling.v_thermal_e_code / 5  # mean
σ = 1.0 #scaling.v_thermal_e_code

exp(- μ^2  / (2 * σ^2) ) / erfc(- μ/σ / sqrt(2) )

9.643749239819589e-23

In [477]:
size(samples,1)

10000

In [691]:
function get_mean_positive(samples)
    sum = 0.
    count = 0

    positive_samples = samples[samples .> 0]
    for i in 1:size(samples,1)
        if samples[i] > 0
            sum += samples[i]
            count += 1
        end
     end
    return sum / count, sum, count, positive_samples
end


function get_mean_negative(samples)
    sum = 0.
    count = 0

    negative_samples = samples[samples .< 0]
    for i in 1:size(samples,1)
        if samples[i] < 0
            sum += samples[i]
            count += 1
        end
     end
    return sum / count, sum, count, negative_samples
end

get_mean_negative (generic function with 1 method)

In [824]:
using Distributions
using SpecialFunctions

# Parameters
n  = 32
μ = 0.0 #scaling.v_thermal_e_code  # mean
σ = 1.0 #scaling.v_thermal_e_code

# Normal distribution
dist = Normal(μ, σ)

# Sample from the distribution
samples = rand(dist, n)

#mean_obs = mean(abs.(samples))
mean_obs, sum, count, positive_samples = get_mean_positive(samples)

mean_theo = σ * sqrt(2 / pi )

#mean_theo_complex = μ + σ * exp(- μ^2  / (2 * σ^2) ) / erfc(- μ/σ / sqrt(2) )

#mean_theo_complex = μ + σ * sqrt(2 / pi) * exp(- μ^2  / (2 * σ^2) ) 
mean_theo_complex = μ + σ * sqrt(2 / pi) * exp(- μ^2  / (2 * σ^2) ) / erfc(- μ/σ / sqrt(2) )


error = abs(mean_obs - mean_theo) / mean_theo * 100
error_complex = abs(mean_obs - mean_theo_complex) / mean_theo_complex * 100

println("mu = ", μ)
println("sigma = ", σ)
println("---")

println("mean_obs = ", mean_obs)
println("mean_theo = ", mean_theo)
println("error = ", error, "%")
println("mean_theo_complex = ", mean_theo_complex)
println("error_complex = ", error_complex, "%")

v_diff = mean_obs - mean_theo_complex

println("v_diff = ", v_diff)
println("--------------------")




new_samples = rand(dist, n)
mean_obs_neg, sum, count_neg, negative_samples = get_mean_negative(new_samples)
#println("mean_obs_neg = ", mean_obs_neg)
#println("count = ", count_neg)
#v_diff_neg = mean_obs_neg + mean_obs#(- (mean_theo_complex - v_diff))
#println("v_diff_neg = ", v_diff_neg)
#
#println("--------------------")
#
#new_samples = new_samples .- v_diff
#mean_obs_neg, sum, count_neg, negative_samples = get_mean_negative(new_samples)
#println("mean_obs_neg = ", mean_obs_neg)
#println("count = ", count_neg)
#v_diff_neg = mean_obs_neg + mean_obs#(- (mean_theo_complex - v_diff))
#println("v_diff_neg = ", v_diff_neg)

while abs(mean_obs_neg + mean_obs) > 1e-6 do
    
end do

#new_samples = new_samples .- v_diff_neg
#mean_obs_neg, sum, count_neg, negative_samples = get_mean_negative(new_samples)
#println("mean_obs_neg = ", mean_obs_neg)
#println("count = ", count_neg)
#v_diff_neg =  mean_obs_neg + mean_obs#(- (mean_theo_complex - v_diff))
#println("v_diff_neg = ", v_diff_neg)


avg = (mean_obs_neg * count_neg + mean_obs * count) / (count + count_neg)
println("avg = ", avg)

mu = 0.0
sigma = 1.0
---
mean_obs = 0.6854687427980806
mean_theo = 0.7978845608028654
error = 14.089233396328318%
mean_theo_complex = 0.7978845608028654
error_complex = 14.089233396328318%
v_diff = -0.11241581800478484
--------------------
mean_obs_neg = -0.7597220900553757
count = 20
v_diff_neg = -0.07425334725729515
--------------------
mean_obs_neg = -0.6834005581637413
count = 19
v_diff_neg = 0.0020681846343392207
avg = -0.05763173486690845


In [827]:
0.0001953125 / scaling.v_thermal_e_code

0.10257185196171992

1.519294282996551

In [27]:
0.5 * scaling.k_B_code * scaling.temperature_code * scaling.weight_scaled * 3 * 32

2.205666317722925e-17

In [28]:
internal_energy_diff = 4.18392090E-18
w_diff = 32

T_diff = 2.  * internal_energy_diff / (scaling.k_B_code * scaling.weight_scaled * w_diff)


sigma = sqrt(2 * internal_energy_diff / ( w_diff  * scaling.weight_scaled_mass_electron))

0.0014364294367181772

In [29]:
scaling.v_thermal_e_code

0.001904153003622194

In [30]:
e_int = 6.78035812E-18 +  3.84699204E-18 +  4.67773490E-18
e_bulk = 1.10867483E-19 +   1.22794267E-17 +  2.37610022E-20
e_tot = e_int + e_bulk

2.77191402452e-17

In [31]:
e_int = 1.01289209E-17 +   4.99518573E-18 +   6.89150969E-18
e_bulk = 3.79406486E-20  +  1.30830964E-17 +  5.01954683E-21
e_tot = e_int + e_bulk

3.514167291543e-17

In [32]:
v_avg =  σ   * sqrt(pi / 2)

tau = 0.5 * ds / v_avg

F = dt / tau

N_out = 0.5 * per_cell * F
N_out_total = N_out * 32 

4.166578855344931

In [33]:
32 / 4.166578855344931

7.680161857239319

In [34]:
t_cross = ds / (σ )  * sqrt(pi / 2)
t_avg = t_cross / 2.


P_esc = 0.5 * per_cell / t_avg 
P_esc_dt = P_esc * dt

boundary_cells = 32

P_esc_dt_tot = P_esc_dt * boundary_cells

2.6525264824412687

In [35]:
(0.27306230987981195 - 0.25641216903348207) / dt

4.9999999999999645

In [36]:
9.3240788739447980E-002 / dt

28.000000000000007

In [37]:
#N = scaling.number_density_code * ds^2 * dt / (sqrt(2 * pi * scaling.m_e_code / (scaling.k_B_code * scaling.temperature_code)))
v_mean = sqrt(8 / pi * scaling.k_B_code * scaling.temperature_code / scaling.m_e_code)

flux_1D = v_mean * scaling.number_density_code / 2
flux_2D = v_mean * scaling.number_density_code / 3
flux_3D = v_mean * scaling.number_density_code / 4

println("flux_1D = ", flux_1D)
println("flux_2D = ", flux_2D)
println("flux_3D = ", flux_3D)


N_1D = flux_1D * ds^2 * dt / scaling.weight_scaled * 1
N_2D = flux_2D * ds^2 * dt / scaling.weight_scaled * 32
N_3D = flux_3D * ds^2 * dt / scaling.weight_scaled * 32

println("N_1D = ", N_1D)
println("N_2D = ", N_2D)
println("N_3D = ", N_3D)

flux_1D = 5.672737844084825e14
flux_2D = 3.781825229389883e14
flux_3D = 2.8363689220424125e14
N_1D = 0.08289145257628965
N_2D = 1.768350988294179
N_3D = 1.3262632412206343


In [73]:
function wave_killing(q, q0, tau, xkill, xbndy, x)
    # Define the variables as floating point numbers
    #-rate::Float64 = 0.0
    #quadfunc::Float64

    # Check the condition to apply the quadratic function
    if abs(x - xbndy) < abs(xkill - xbndy)
        quadfunc = ((x - xkill) / (xbndy - xkill))^2
        rate = -(q - q0) * quadfunc / tau
    else
        rate = 0.0
    end

    return rate
end


wave_killing (generic function with 1 method)

In [142]:
0.1 / dt

7.5075075075075075

In [145]:
Bx_flux_code = 1.0 #scaling.B_flux_code
dt = 1.3320E-02


Bx_flux_test = Bx_flux_code * 0.9
tau = 0.1#!20 * dt

#x_start = 1 - 0.0625
#x_end   = 1.0
#x = 1 - ds 

x_start = 0.0625
x_end   = - ds * 3
x =  +3 *  ds 

for i in 1:30
    rate = wave_killing(Bx_flux_test, Bx_flux_code, tau, x_start, x_end, x)
    Bx_flux_test = Bx_flux_test + rate * dt 
    println("Bx_flux_test = ", Bx_flux_test)
end 

Bx_flux_test = 0.9091445877551021
Bx_flux_test = 0.9174529406580965
Bx_flux_test = 0.925001528938873
Bx_flux_test = 0.9318598299400426
Bx_flux_test = 0.9380909675876511
Bx_flux_test = 0.9437522933849329
Bx_flux_test = 0.94889591427658
Bx_flux_test = 0.9535691722420008
Bx_flux_test = 0.9578150800317513
Bx_flux_test = 0.9616727170576673
Bx_flux_test = 0.9651775890804752
Bx_flux_test = 0.9683619550054534
Bx_flux_test = 0.9712551237939784
Bx_flux_test = 0.9738837242237335
Bx_flux_test = 0.9762719499804586
Bx_flux_test = 0.9784417823370701
Bx_flux_test = 0.9804131924696926
Bx_flux_test = 0.9822043252727245
Bx_flux_test = 0.9838316663647727
Bx_flux_test = 0.9853101938225838
Bx_flux_test = 0.986653516039532
Bx_flux_test = 0.9878739969775177
Bx_flux_test = 0.9889828699650949
Bx_flux_test = 0.9899903410892305
Bx_flux_test = 0.9909056831323122
Bx_flux_test = 0.9917373209190049
Bx_flux_test = 0.992492908858489
Bx_flux_test = 0.9931794013957799
Bx_flux_test = 0.993803117020566
Bx_flux_test = 0.994

In [155]:
scaling.mass_density_code * scaling.rho_n_ratio * scaling.k_B_code * scaling.temperature_code / (gamma - 1.0)

4.933992031514952e-9